<a href="https://colab.research.google.com/github/Gmarco2706/DNAGYMCraft/blob/main/GymGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Caricamento del nuovo dataset
gym_dataset2 = pd.read_csv('gym_dataset2.csv')

gym_dataset2

In [ ]:
print(gym_dataset2.columns)

In [ ]:
#input
livello_di_abilita = input("Inserisci il tuo livello di abilità (Principiante/Intermedio/Esperto): ")
luogo_di_allenamento = input("Ti alleni in palestra o a casa? (Palestra/Casa): ")

fitness_storico = []


<h1>Creazione e inizializzazione della popolazione </h1>

<p>
creiamo una popolazione iniziale di schede di allenamento, ciascuna con una struttura che include giorni settimanali e esercizi casuali selezionati dal dataset. La funzione <b>generate_random_workout</b> genera una scheda di allenamento casuale con 7 esercizi rispettando il vincolo di avere un esercizio di stretching e di avere un max di 7 esercizi al giorno compreso lo stretching. La funzione <b>generate_initial_population</b> crea una popolazione di schede di allenamento settimanali per gli individui. Alla fine stampiamo la prima scheda
</p>

In [ ]:
#creazione della popolazione
def generate_random_workout(data, max_exercises=7, include_stretching=True):

    workout = []
    #estrazione gli esercizi di stretching dal dataset
    stretching_exercises = data[data['Type_Stretching'] == 1]

    #aggiunge un esercizio di stretching random (vincolo)
    if include_stretching and not stretching_exercises.empty:
        workout.append(stretching_exercises.sample(n=1).iloc[0])

    remaining_slots = max_exercises - len(workout)
    #estrazione degli esercizi che non sono di stretching
    other_exercises = data[data['Type_Stretching'] == 0]
    #filla tutti gli slot vuoti rimanenti (vincolo max 7 es)
    workout += other_exercises.sample(n=min(len(other_exercises), remaining_slots)).to_dict('records')

    return workout
#genera la prima generazione di schede di allenamento
def generate_initial_population(data, population_size=10, days_per_week=3):
    population = []
    #per ogni individuo crea schede di allenamento
    for _ in range(population_size):
        weekly_schedule = {}
        for day in range(days_per_week):
            daily_workout = generate_random_workout(data)
            weekly_schedule[f"Day {day + 1}"] = daily_workout
        population.append(weekly_schedule)

    return population

# Generiamo una popolazione iniziale di schede di allenamento
initial_population = generate_initial_population(gym_dataset2)
initial_population[0]  # Visualizziamo la scheda



<h1>Funzione di Fitness</h1>


<p>
creiamo  la funzione di fitness che valuta una scheda di allenamento in base a vari criteri: (1)livello di abilità, (2)varietà di tipi di esercizi, (3)la varietà di gruppi muscolari coinvolti, (4)il luogo di allenamento, (5)feedback fornito.
La funzione calcola un punteggio totale in base a questi criteri e lo restituisce come risultato.
</p>

In [ ]:
#funzione di fitness
def fitness(scheda, livello_di_abilita, luogo_di_allenamento, pesi, feedback = None):
    punteggio = 0
    #creo i vari livelli di abilità associandoli alle colonne del dataset
    livello_mapping = {"Principiante": "Level_Beginner", "Intermedio": "Level_Intermediate", "Esperto": "Level_Expert"}
    livello_colonna = livello_mapping[livello_di_abilita]
    #iniziamo a calcolare il punteggio basandoci sull'aderenza al livello di abolità dell'utente
    for day in scheda:
        for esercizio in scheda[day]:
            if esercizio[livello_colonna] == 1:
                punteggio += pesi['livello_aderenza']
    #calcoliamo il punteggio sta volta sulla base della differenziazione degli esercizi e dei gruppi muscolari
    tipi_esercizi = set()
    gruppi_muscolari = set()
    for day in scheda:
        for esercizio in scheda[day]:
            for tipo in ["Type_Cardio", "Type_Strength", "Type_Stretching"]:
                if esercizio[tipo] == 1:
                    tipi_esercizi.add(tipo)
            for gruppo in ["BodyPart_Abdominals", "BodyPart_Biceps", "BodyPart_Calves", "BodyPart_Chest", "BodyPart_Forearms", "BodyPart_Glutes", "BodyPart_Hamstrings", "BodyPart_Lats", "BodyPart_Lower Back", "BodyPart_Middle Back", "BodyPart_Neck", "BodyPart_Quadriceps", "BodyPart_Shoulders", "BodyPart_Traps", "BodyPart_Triceps"]:
                if esercizio.get(gruppo, 0) == 1:
                    gruppi_muscolari.add(gruppo)

    punteggio += len(tipi_esercizi) * pesi['varietà_tipo'] + len(gruppi_muscolari) * pesi['varietà_gruppo']
    #andiamo a modificare il punteggio in base all'input del luogo di allenamento scegliendo gli esercizi senza attrezzi per chi si allena da casa
    if luogo_di_allenamento == "Casa":
        for day in scheda:
            for esercizio in scheda[day]:
                if esercizio["Equipment_None"] == 1:
                    punteggio += pesi['attrezzatura_casa']


    # Modifica del punteggio in base al feedback
    if feedback:
        if feedback == "Ottima":
            punteggio += 10  # Incoraggia schede simili
        elif feedback == "Troppo intensa":
            punteggio -= 5  # Penalizza schede troppo intense
        elif feedback == "Troppo facile":
            punteggio -= 6 #penalizza schede troppo facili

    return punteggio
